In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
text = open("input.txt", 'r').read()
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [14]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(vocab_size, ''.join(chars))

65 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [4]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
print(itos)

{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i', 48: 'j', 49: 'k', 50: 'l', 51: 'm', 52: 'n', 53: 'o', 54: 'p', 55: 'q', 56: 'r', 57: 's', 58: 't', 59: 'u', 60: 'v', 61: 'w', 62: 'x', 63: 'y', 64: 'z'}


In [5]:
encode = lambda s: [stoi[ch] for ch in s] # take a string and output a list of integers
decode = lambda l: ''.join([itos[i] for i in l])

In [6]:
print(encode("hi there !!"))
print(decode(encode("hi there !!")))

[46, 47, 1, 58, 46, 43, 56, 43, 1, 2, 2]
hi there !!


In [16]:
data = torch.tensor(encode(text), dtype=torch.long)
data.dtype

torch.int64

In [17]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [18]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [19]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when the input is {context}: the target is {target}")

when the input is tensor([18]): the target is 47
when the input is tensor([18, 47]): the target is 56
when the input is tensor([18, 47, 56]): the target is 57
when the input is tensor([18, 47, 56, 57]): the target is 58
when the input is tensor([18, 47, 56, 57, 58]): the target is 1
when the input is tensor([18, 47, 56, 57, 58,  1]): the target is 15
when the input is tensor([18, 47, 56, 57, 58,  1, 15]): the target is 47
when the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]): the target is 58


In [11]:
# torch.randint(len(data)-block_size, (batch_size,))

In [20]:
# generalizing it for a batch of data
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    # print([data[i:i+block_size] for i in ix])
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb,yb = get_batch("train")
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)
print("----------------")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1] #batch_size, t
        target = yb[b, t] #batch_size,1
        print(f"when input is {context.tolist()} ----- the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----------------
when input is [24] ----- the target: 43
when input is [24, 43] ----- the target: 58
when input is [24, 43, 58] ----- the target: 5
when input is [24, 43, 58, 5] ----- the target: 57
when input is [24, 43, 58, 5, 57] ----- the target: 1
when input is [24, 43, 58, 5, 57, 1] ----- the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] ----- the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] ----- the target: 39
when input is [44] ----- the target: 53
when input is [44, 53] ----- the target: 56
when input is [44, 53, 56] ----- the target: 1
when input is [44, 53, 56, 1] ----- t

In [25]:
# define and evaluate the model:

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets):
        logits = self.token_embedding_table(idx) # (B,T,C=vocab_size)
        
        # loss = F.cross_entropy(logits, targets) # RuntimeError: Expected target size [4, 65], got [4, 8]
        B,T,C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)

        return logits, loss
    
m = BigramLanguageModel()
logits, loss = m(xb, yb)
print(logits.shape, loss)

torch.Size([32, 65]) tensor(4.5262, grad_fn=<NllLossBackward0>)


In [27]:
# expected initial loss is around ?
-torch.log(torch.tensor(1/65))

tensor(4.1744)

In [31]:
# including generation from the model

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # (B,T,C=vocab_size)
        if targets is None:
            loss = None
        else:         
            # loss = F.cross_entropy(logits, targets) # RuntimeError: Expected target size [4, 65], got [4, 8]
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        # job of generate is to extend it to (B,T+1), (B,T+2), (B,T+3) ... and so on; upto max_new_tokens
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:,-1,:] # (B,C); focussing only on the last time-step channels as it is BIGRAM lm.
            probs = F.softmax(logits, dim=-1) # (B,C)
            idx_next = torch.multinomial(probs, num_samples=1) # (B,1)
            idx = torch.cat((idx,idx_next), dim=1) # (B,T+1)
        return idx

m = BigramLanguageModel()
logits, loss = m(xb, yb)
print(logits.shape, loss)

# input for starting generation
idx = torch.zeros((1,1), dtype=torch.long) # B=1,T=1, 0 - is to start with newline character
print(decode(m.generate(idx, max_new_tokens=200)[0].tolist()))

torch.Size([32, 65]) tensor(4.7369, grad_fn=<NllLossBackward0>)

U;FSmqMVd&
csL:z&lhI$OLyS,oNTS,;!Us;yLZr'u?RytaluC,iKfYjG,l3jWHJy:hqM;caMmTJ,?zlukJ!Gk!LXG,tE k:$zmpf:R&ejQCgBSNhI::bBHMR'-YQzqk3ki;.eXGXAvrd&xp,Tw'UmYoht?zuheWHgjlq;3hDvsl$ezhL Zj-ZKUj$UURfMMpfbVJ,MV


In [34]:
# training the model

optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

batch_size = 32
for steps in range(10000):
    xb,yb = get_batch("train")
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=None)
    loss.backward()
    optimizer.step()

print(loss.item())

2.533872365951538


In [36]:
idx = torch.zeros((1,1), dtype=torch.long) # B=1,T=1, 0 - is to start with newline character
print(decode(m.generate(idx, max_new_tokens=500)[0].tolist()))


ARTofter esthon m, in fegoino pad t

Swn, Thathes toecimPulou o tof maindome m, nomo;
Hathmeporbrderse t wind whrde fl w.
hametespr, athen, henghead,
by t,
Floulivedig che ay tink'GA:


MBEETo he w wede he ate s y wo u tho.
Wern tou wethicke, gh he wis?


Ge owony ctwhomyied shin d o ubusthy ne, Asehen boumpr t d ke e bee t hirm w mands so'sey sothoul mn n wofirw ho'ds ar?
R: ls th, myon!
WI Fr mby

G;
t the ermy yong
ALUthe.
Sove. noul y,
HO:
An nothyillly,
Sind h myourt r itonf plin
Thy al n, 
